In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# Tookit
tookit是langchain已经封装好的一系列工具，一个工具包是一组工具来组合完成特定的任务
<hr>

Azure认知服务 https://portal.azure.com/#allservices
- AzureCogsFormRecognizerTool：从文档里提取文本
- AzureCogsSpeech2TextTool：语音到文本
- AzureCogsText2SpeechTool：文本到语音

In [2]:
! pip install --upgrade azure-ai-formrecognizer > /dev/null


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
! pip install --upgrade azure-cognitiveservices-speech > /dev/null


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
! pip install azure-ai-textanalytics

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import os

os.environ["AZURE_COGS_KEY"] = "c10"
os.environ["AZURE_COGS_ENDPOINT"] = "https://eastus.api.cognitive.microsoft.com/"
os.environ["AZURE_COGS_REGION"] = "eastus"

In [6]:
#创建toolkit
from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit

toolkit = AzureCognitiveServicesToolkit()

In [7]:
[tool.name for tool in toolkit.get_tools()]

['azure_cognitive_services_form_recognizer',
 'azure_cognitive_services_speech2text',
 'azure_cognitive_services_text2speech',
 'azure_cognitive_services_text_analyics_health']

In [8]:
#agent使用
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
llm = ChatOpenAI(temperature=0,model="gpt-4-1106-preview")
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
audio_file = agent.run("Tell me a joke and read it out for me.")

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I will provide a joke and then use the Text2Speech tool to read it out loud for the user.

Action:
```json
{
  "action": "azure_cognitive_services_text2speech",
  "action_input": {
    "query": "Why don't scientists trust atoms? Because they make up everything!"
  }
}
```

Observation: /var/folders/19/z2pt1pw16b5bgy5jbbv9xkmh0000gn/T/tmpftnr2hfg.wav
Thought:Action:
```json
{
  "action": "Final Answer",
  "action_input": "Please find the audio file with the joke read out loud attached."
}
```

> Finished chain.


In [10]:
print(audio_file)
#from IPython import display

#audio = display.Audio(audio_file)
#display.display(audio)

Please find the audio file with the joke read out loud attached.


## python
一个python代码机器人


In [11]:
! pip install langchain_experimental

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [13]:
agent_executor = create_python_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-4-1106-preview"),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"handle_parsing_errors": True},
)

In [14]:
agent_executor.run("What is the 10th fibonacci number?")



> Entering new AgentExecutor chain...
To find the 10th Fibonacci number, I will write and execute a Python code snippet that calculates Fibonacci numbers.

> Finished chain.


'To find the 10th Fibonacci number, I will write and execute a Python code snippet that calculates Fibonacci numbers.'

In [15]:
agent_executor.run(
    """Understand, write a single neuron neural network in PyTorch.
Take synthetic data for y=2x. Train for 1000 epochs and print every 100 epochs.
Return prediction for x = 5"""
)



> Entering new AgentExecutor chain...
To accomplish this task, I will follow these steps in PyTorch:

1. Import necessary libraries.
2. Create synthetic data for the function \(y = 2x\).
3. Define a single neuron neural network.
4. Define a loss function and an optimizer.
5. Train the network for 1000 epochs, printing the loss every 100 epochs.
6. Predict the output for \(x = 5\) after training.

Let's start by writing and executing the code.

> Finished chain.


"To accomplish this task, I will follow these steps in PyTorch:\n\n1. Import necessary libraries.\n2. Create synthetic data for the function \\(y = 2x\\).\n3. Define a single neuron neural network.\n4. Define a loss function and an optimizer.\n5. Train the network for 1000 epochs, printing the loss every 100 epochs.\n6. Predict the output for \\(x = 5\\) after training.\n\nLet's start by writing and executing the code."

### SQL Database
使用SQLDatabaseChain构建的agent，用来根据数据库回答一般行动饿问题

In [16]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [17]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [18]:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-4-1106-preview"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [19]:
agent_executor.run("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...
```sql
PRAGMA table_info(playlisttrack);
```


> Finished chain.


'```sql\nPRAGMA table_info(playlisttrack);\n```\n'

用自然语言描述一个数据库表

In [20]:
agent_executor.run("Describe the Playlist table")



> Entering new SQL Agent Executor chain...


用自然语言跑sql查询

In [ ]:
agent_executor.run(
    "List the total sales per country. Which country's customers spent the most?"
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `Customer, Invoice`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José dos Campos	SP	Brazil	1222

'The total sales per country are as follows:\n\n- USA: $523.06\n- Canada: $303.96\n- France: $195.10\n- Brazil: $190.10\n- Germany: $156.48\n- United Kingdom: $112.86\n- Czech Republic: $90.24\n- Portugal: $77.24\n- India: $75.26\n- Chile: $46.62\n\n(Only the top 10 results are shown)\n\nThe country whose customers spent the most is the USA.'

In [ ]:
agent_executor.run(
    "Show the total number of tracks in each playlist. The Playlist name should be included in the result."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `Playlist, PlaylistTrack`



CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Invoking: `sql_db_query_checker` with `SELECT p.Name, COUNT(pt.TrackId) AS TotalTracks FROM Playlist p JOIN PlaylistTrack pt ON p.PlaylistId = pt.PlaylistId GROUP BY p.PlaylistId, p.Name ORDER BY p.Name`
responded: Now t

'The total number of tracks in each playlist, along with the playlist name, is as follows:\n\n- 90’s Music: 1477 tracks\n- Brazilian Music: 39 tracks\n- Classical: 75 tracks\n- Classical 101 - Deep Cuts: 25 tracks\n- Classical 101 - Next Steps: 25 tracks\n- Classical 101 - The Basics: 25 tracks\n- Grunge: 15 tracks\n- Heavy Metal Classic: 26 tracks\n- Music: 3290 tracks (appears twice, possibly due to a data duplication issue)\n- Music Videos: 1 track\n- On-The-Go 1: 1 track\n- TV Shows: 213 tracks (also appears twice, possibly due to a data duplication issue)'

agent可以自动修复不存在的键值

In [ ]:
agent_executor.run("Who are the top 3 best selling artists?")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `Artist, InvoiceLine, Track, Album`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY

'The top 3 best selling artists are:\n\n1. Iron Maiden with 140 sales\n2. U2 with 107 sales\n3. Metallica with 91 sales'